In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

/var/folders/tp/yn0jfb7970b33_f1g1644dn40000gp/T/ipykernel_4707/2004128412.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
def fetch_books_from_page(page):
    data = []
    url = 'https://www.goodreads.com/review/list/58496717-kathrin?print=true&shelf=to-read&utf8=%E2%9C%93&page=' + str(page)
    response = requests.get(url)
    html = response.content

    soup = BeautifulSoup(html, 'html.parser')


    rows = soup.find_all('tr', {'class': 'bookalike review'})

    for row in rows:
        # Find the fields
        title_field = row.find('td', {'class': 'field title'})
        author_field = row.find('td', {'class': 'field author'})
        date_pub_field = row.find('td', {'class': 'field date_pub'})

        # Extract the text
        title = title_field.find('div', {'class': 'value'}).text.strip()
        author = author_field.find('div', {'class': 'value'}).text.strip()
        date_pub = date_pub_field.find('div', {'class': 'value'}).text.strip()

        # Add to the list
        data.append([title, author, date_pub])

    return data

In [5]:
data = []

page = 1
while True:
    new_data = fetch_books_from_page(page)
    if len(new_data) == 0:
        break
    data += new_data
    page += 1

df = pd.DataFrame(data, columns=["title", "author", "publication_date"])



In [6]:
df_clone = df.copy()

df_clone["publication_date"] = pd.to_datetime(df_clone["publication_date"], errors = "coerce")

df_sorted = df_clone.sort_values(by = ["publication_date"], ascending=False)


/var/folders/tp/yn0jfb7970b33_f1g1644dn40000gp/T/ipykernel_4707/1276229535.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_clone["publication_date"] = pd.to_datetime(df_clone["publication_date"], errors = "coerce")


,title,author,publication_date
100,"Where Shadows Meet\n (Heirs of Shadow, #1)","Caldwell, Patrice\n*",2024-10-01
7,Ghostsmith (2)\n (2) (House of the Dead...,"Pau Preto, Nicki\n*",2024-08-13
123,"The Half King\n (The Half King, #1)","Landers, Melissa\n*",2024-07-09
96,The Darkness Within Us,"Levenseller, Tricia\n*",2024-07-09
11,Five Broken Blades,"Corland, Mai",2024-05-07
68,The Dead Cat Tail Assassins,"Clark, P. Djèlí\n*",2024-04-23
65,Song of the Six Realms,"Lin, Judy I.\n*",2024-04-23
39,To Gaze Upon Wicked Gods\n (To Gaze Upo...,"Chang, Molly X.\n*",2024-04-16
71,The Hemlock Queen\n (The Nightshade Cro...,"Whitten, Hannah\n*",2024-04-09
38,The Last Phi Hunter,"Goldenberg, Salinee\n*",2024-04-09


In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Your email credentials
sender_email = "email@gmail.com"
receiver_email = "email@gmail.com"  
password = "1234"

# Email content
subject = "List of Books"
body = df_filtered.to_string()

# Setting up the email message
message = MIMEMultipart()
message["From"] = sender_email
message["To"] = receiver_email
message["Subject"] = subject
message.attach(MIMEText(body, "plain"))

# Sending the email
try:
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()  # Secure the connection
    server.login(sender_email, pwd)
    server.sendmail(sender_email, receiver_email, message.as_string())
    print("Email sent successfully!")
except Exception as e:
    print(f"Error sending email: {e}")
finally:
    server.quit()